In [4]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from binance import Client
import pandas as pd

#Config
import sys
sys.path.insert(0,"/Users/marvinottersberg/Documents/GitHub/sentiment/")
from config import ConfigBinance
conf = ConfigBinance().getKeys()

client = Client(conf[0],conf[1])

plt.style.use("ggplot")
print(plt.style.available)

asset = "BTCUSDT"



['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


In [5]:
def getminutedata(symbol,interval, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol,interval,lookback))
    frame = frame.iloc[:,:6]
    frame.columns= ["Time","Open","High","Low","Close","Volume"]
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index,unit = "ms")
    frame = frame.astype(float)
    return frame


In [6]:
df = getminutedata(asset, "1m","120m") #data every 1 minute for total lookback of 120 minutes

df


,Open,High,Low,Close,Volume
Time,,,,,
2022-07-13 09:26:00,19867.97,19884.98,19867.82,19883.52,95.85977
2022-07-13 09:27:00,19883.52,19887.95,19873.76,19876.26,94.58965
2022-07-13 09:28:00,19874.45,19896.47,19873.35,19893.10,112.96804
2022-07-13 09:29:00,19893.10,19899.64,19887.03,19891.72,66.64642
2022-07-13 09:30:00,19894.21,19898.38,19867.00,19867.92,69.71485
...,...,...,...,...,...
2022-07-13 11:21:00,19797.10,19801.00,19792.69,19794.79,39.96429
2022-07-13 11:22:00,19796.27,19815.46,19794.79,19812.32,152.93875
2022-07-13 11:23:00,19812.32,19814.98,19795.37,19795.60,156.26336


In [22]:
def animate(i):
    data = getminutedata(asset,"1m","120m")
    #plt.figure(figsize=((10,8)))
    plt.cla()
    plt.plot(data.index,data.Close)
    plt.xlabel("Time")
    plt.ylabel("Price")
    plt.title(asset)
    plt.gcf().autofmt_xdate() #adjust dates 
    plt.tight_layout()
    

In [23]:
def getPlot():
    ani = FuncAnimation(plt.gcf(),animate,1000) #updating every second
    plt.figure(figsize=((10,8)))
    plt.tight_layout()
    plt.show()
    return ani

anim = getPlot()

<Figure size 432x288 with 0 Axes>

<Figure size 720x576 with 0 Axes>